In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
class CFG:
    fea_dir = '../input/ubiquant-train-data/train_std_data.npy'
    target_dir = '../input/ubiquant-train-data/train_(timeid)target-f32.npy'
    tfrecords_dir = '../input/ubiquant-train-data/train_data_target-f32-all.tfrecords'
    time_dir = None
    inv_dir = None

    input_dim = 300
    enco_noise_rate = 0.05
    
    full_train = True
    
    just_train_enco = False
    freeze_enco = False
    enco_weights_dir = '../input/encoderonly-feaall-data/cur-best-encoder.h5'
    
    mlp_batch_size = 512
    enco_batch_size = 512
    test_batch_size = 4096
    train_shuffle_seed = 113
    
    split_n = 6
    use_time_kfold = True
    t_kfold_start = 4
    simple_kfold_get = 2
    simple_kfold_seed = 125

    mlp_LR = 0.001
    enco_LR = 0.001

    cb_monitor = 'val_loss'
    cb_rlrp_patience = 8
    cb_rlrp_factor = 0.6
    cb_es_patience = 15
    
    mlp_epoch = 12
    enco_epoch = 105
    
    fulltr_savedir = './fulltr-bigdrop-origin-bt512-test-1-ep{epoch}-loss{loss:.4f}.h5'

    def save_mlp_dir(self, the_fold):
        return './origin-bigdrop-test-1_{0}.h5'.format(the_fold)

    def save_encoder_dir(self):
        return './fulltr-enco256256-{0}.h5'.format('full-train-unfreeze-1')

cfg = CFG()


In [3]:
import gc
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import scipy as sp

In [4]:
# if not cfg.full_train:
fea_data = np.load(cfg.fea_dir)
target = np.load(cfg.target_dir)
print(fea_data.shape, fea_data.dtype)
print(target.shape, target.dtype)

if cfg.time_dir is not None:
    time_id = np.load(cfg.time_dir)
if cfg.inv_dir is not None:
    inv_id = np.load(cfg.inv_dir)

(3141410, 300) float16
(3141410, 2) float32


In [5]:
class Mish(layers.Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)

    def call(self, inputs, *args, **kwargs):
        return inputs * K.tanh(K.softplus(inputs))

In [6]:
def PearsonCorrelation(y_true, y_pred):
    yt_mean = K.mean(y_true)
    yp_mean = K.mean(y_pred)
    cor = K.mean((y_true - yt_mean) * (y_pred - yp_mean))
    yt_std = K.std(y_true)
    yp_std = K.std(y_pred)
    pearson = cor / (yt_std * yp_std)
    return pearson

def PearsonCorrelationLoss(y_true, y_pred):
    return K.constant(1.0, dtype=y_pred.dtype) - PearsonCorrelation(y_true, y_pred)

def MyLoss(y_true, y_pred):
    return losses.mean_squared_error(y_true, y_pred) * 0.5 + PearsonCorrelationLoss(y_true, y_pred) * 0.5

In [7]:
def build_encoder(input_dim, noise_rate=0.05):
    ori_input = Input(shape=(input_dim,), dtype='float16')
    encode = layers.BatchNormalization()(ori_input)
    encode = layers.GaussianNoise(stddev=noise_rate)(encode)
    encode = layers.Dense(256)(encode)
    encode = Mish()(encode)
    decode = layers.Dropout(0.1)(encode)
    decode = layers.Dense(input_dim, name='decoded')(decode)
    x = layers.Dense(265, use_bias=False)(decode)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(1, name='target')(x)

    encoder = models.Model(inputs=[ori_input], outputs=[encode])
    autoencoder = models.Model(inputs=[ori_input], outputs=[decode, x])
    autoencoder.compile(optimizer=optimizers.Adam(learning_rate=cfg.enco_LR),
                        loss={'decoded': losses.MeanSquaredError(),
                              'target': losses.MeanSquaredError()},
                        metrics={'target': PearsonCorrelation})
    return autoencoder, encoder

In [8]:
def build_model(encoder):
    ox_input = Input(shape=(cfg.input_dim,), dtype='float16')
    ox_enco = encoder(ox_input)

    x = layers.Dense(512, use_bias=False)(ox_input)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128)(x)
    x = Mish()(x)

    x = layers.Concatenate()([x, ox_enco])
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = Mish()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    model = models.Model(inputs=[ox_input], outputs=[output])
    model.compile(optimizer=optimizers.Adam(learning_rate=cfg.mlp_LR),
                  loss=losses.MeanSquaredError(),
                  metrics=[PearsonCorrelation])
    return model

In [9]:
def see_model_config(my_model):
    print(my_model.summary())
    plot_model(my_model, './the_model.png')

In [10]:
def metrics_ave_time_id(y_true, y_pred):
    person_list = []
    y_pred = np.array(y_pred)
    y_pred = y_pred.flatten()
    time_id = y_true[:, 0]
    y_true = y_true[:, 1]
    for t in np.unique(time_id):
        y_t = y_true[time_id == t]
        if y_t.shape[0] == 1:
            continue
        y_p = y_pred[time_id == t]
        y_t_mean = np.average(y_t)
        y_p_mean = np.average(y_p)
        cov = np.average((y_t - y_t_mean) * (y_p - y_p_mean))
        y_t_std = np.std(y_t)
        y_p_std = np.std(y_p)
        person_list.append(cov / (y_t_std * y_p_std + 1e-5))
    person_list = np.array(person_list)
    return np.average(person_list), np.average(person_list[person_list >= 0.0])

In [11]:
def make_dataset(feature, y, batch_size=512, mode="train", to='mlp'):
    if to == 'mlp':
        ds = tf.data.Dataset.from_tensor_slices((feature, y))
    else:
        ds = tf.data.Dataset.from_tensor_slices((feature, (feature, y)))
    if mode == "train":
        ds = ds.shuffle(4096, seed=cfg.train_shuffle_seed, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [12]:
def decode_function(record_bytes):
    return tf.io.parse_single_example(record_bytes,
                                      {"feature": tf.io.FixedLenFeature([300], dtype=tf.float32),
                                        "target": tf.io.FixedLenFeature([], dtype=tf.float32)})

def preprocess_mlp(item):
    return (item["feature"]), item["target"]

def preprocess_enco(item):
    return (item["feature"]), (item['feature'], item["target"])

def make_fulltr_dataset(file_path, batch_size=512, to='mlp'):
    ds = tf.data.TFRecordDataset(file_path)
    ds = ds.map(decode_function)
    if to == 'mlp':
        ds = ds.map(preprocess_mlp)
    else:
        ds = ds.map(preprocess_enco)
    ds = ds.shuffle(4096, seed=cfg.train_shuffle_seed, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

In [13]:
def get_encoder(mode='new', train_ds=None, test_ds=None, the_callback=None):
    if mode == 'freeze':
        autoencoder, encoder = build_encoder(cfg.input_dim)
        encoder.load_weights(cfg.enco_weights_dir)
    else:
        assert the_callback != None, 'callbacks 不能是 None!'
        autoencoder, encoder = build_encoder(cfg.input_dim, noise_rate=cfg.enco_noise_rate)
        print(encoder.summary())
        if mode == 'new':
            autoencoder.fit(train_ds,
                            epochs=cfg.enco_epoch,
                            callbacks=the_callback)
            encoder.save_weights(cfg.save_encoder_dir())
        elif mode == 'cv':
                autoencoder.fit(train_ds,
                                validation_data=test_ds,
                                epochs=cfg.enco_epoch,
                                callbacks=the_callback,
                                verbose=2)
    encoder.trainable = False
    return encoder

In [14]:
if cfg.full_train:
    if cfg.freeze_enco:
        encoder = get_encoder(mode='freeze')
    else:
        callback = [callbacks.ReduceLROnPlateau(monitor='loss', patience=8, factor=0.55, verbose=1),
                    callbacks.EarlyStopping(monitor='loss', patience=16, restore_best_weights=True, min_delta=0.0012)]
        train_ds = make_dataset(fea_data, target[:, 1], batch_size=cfg.enco_batch_size, to='encoder')
        encoder = get_encoder(mode='new', train_ds=train_ds, the_callback=callback)
        del train_ds
        gc.collect()
        
    callback = [callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, 
                                            patience=cfg.cb_rlrp_patience, 
                                            factor=cfg.cb_rlrp_factor),
                callbacks.ModelCheckpoint(cfg.fulltr_savedir,
                                          monitor='loss', save_freq='epoch',
                                          save_best_only=False,
                                          save_weights_only=True)]
    print('训练mlp**********************************************************************************************')
    the_model = build_model(encoder)
    see_model_config(the_model)
    train_ds = make_dataset(fea_data, target[:, 1], batch_size=cfg.mlp_batch_size)
    the_model.fit(train_ds,
                  epochs=cfg.mlp_epoch,
                  callbacks=callback,
                  verbose=1)
    
    ave_person_score, pos_ave_person_score = metrics_ave_time_id(target, the_model.predict(fea_data, batch_size=2048))
    print('\n Ave_Person_Score: {0}\n Positive_Person_Score: {1}\n'.format(ave_person_score, pos_ave_person_score))
    

2022-04-14 02:38:28.660634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:38:28.770249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:38:28.771050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 02:38:28.773975: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 300)               1200      
_________________________________________________________________
gaussian_noise (GaussianNois (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               77056     
_________________________________________________________________
mish (Mish)                  (None, 256)               0         
Total params: 78,256
Trainable params: 77,656
Non-trainable params: 600
_________________________________________________________________
None


2022-04-14 02:38:40.867307: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1884846000 exceeds 10% of free system memory.


Epoch 1/105


2022-04-14 02:38:46.574252: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


6136/6136 [==============================] - 36s 5ms/step - loss: 1.0874 - decoded_loss: 0.2593 - target_loss: 0.8281 - target_PearsonCorrelation: 0.1808
Epoch 2/105
6136/6136 [==============================] - 34s 6ms/step - loss: 0.9825 - decoded_loss: 0.1691 - target_loss: 0.8134 - target_PearsonCorrelation: 0.1937
Epoch 3/105
6136/6136 [==============================] - 34s 5ms/step - loss: 0.9772 - decoded_loss: 0.1635 - target_loss: 0.8137 - target_PearsonCorrelation: 0.1931
Epoch 4/105
6136/6136 [==============================] - 34s 6ms/step - loss: 0.9748 - decoded_loss: 0.1617 - target_loss: 0.8131 - target_PearsonCorrelation: 0.1941
Epoch 5/105
6136/6136 [==============================] - 33s 5ms/step - loss: 0.9732 - decoded_loss: 0.1609 - target_loss: 0.8123 - target_PearsonCorrelation: 0.1962
Epoch 6/105
6136/6136 [==============================] - 35s 6ms/step - loss: 0.9725 - decoded_loss: 0.1603 - target_loss: 0.8122 - target_PearsonCorrelation: 0.1967
Epoch 7/105
6136

In [15]:
if not cfg.full_train:
    if cfg.just_train_enco:
        train_ds = make_dataset(fea_data, target[:, 1], batch_size=cfg.enco_batch_size, to='encoder')
        callback = [callbacks.ReduceLROnPlateau(monitor='loss', patience=8, factor=0.55, verbose=1),
                    callbacks.EarlyStopping(monitor='loss', patience=18, restore_best_weights=True, min_delta=0.001)]
        get_encoder(mode='new', train_ds=train_ds, the_callback=callback)

    else:
        if cfg.freeze_enco:
            encoder = get_encoder(mode='freeze')
            
        if cfg.use_time_kfold:
            kfold = TimeSeriesSplit(n_splits=cfg.split_n)
        else:
            kfold = KFold(n_splits=cfg.split_n, shuffle=True, random_state=cfg.simple_kfold_seed)

        for fold, (train_ind, test_ind) in enumerate(kfold.split(fea_data, target)):

            if cfg.use_time_kfold:
                if fold < cfg.t_kfold_start:
                    continue

            print('训练集大小：{0:}'.format(len(train_ind)))
            print('测试集大小：{0:}'.format(len(test_ind)))
            train_ox = fea_data[train_ind]
            test_ox = fea_data[test_ind]
            train_target = target[train_ind]
            test_target = target[test_ind]
            
             
            callback = [callbacks.ReduceLROnPlateau(monitor=cfg.cb_monitor,
                                                    patience=cfg.cb_rlrp_patience,
                                                    factor=cfg.cb_rlrp_factor, verbose=1),
                        callbacks.EarlyStopping(monitor=cfg.cb_monitor,
                                                patience=cfg.cb_es_patience,
                                                restore_best_weights=True)]   

            if not cfg.freeze_enco:
                train_ds = make_dataset(train_ox, train_target[:, 1], batch_size=cfg.enco_batch_size, to='encoder')
                test_ds = make_dataset(test_ox, test_target[:, 1], batch_size=cfg.test_batch_size, mode='valid', to='encoder')
                print('训练encoder')
                encoder = get_encoder(mode='cv', train_ds=train_ds, test_ds=test_ds, the_callback=callback)
                del train_ds, test_ds
                gc.collect()
                print('\n\n')
            
            the_model = build_model(encoder)

            if cfg.use_time_kfold:
                if fold == cfg.t_kfold_start:
                    see_model_config(the_model)
            if not fold:
                see_model_config(the_model)

            train_ds = make_dataset(train_ox, train_target[:, 1], batch_size=cfg.mlp_batch_size)
            test_ds = make_dataset(test_ox, test_target[:, 1], batch_size=cfg.test_batch_size, mode='valid')
            the_model.fit(train_ds,
                          epochs=cfg.mlp_epoch,
                          callbacks=callback,
                          validation_data=test_ds,
                          verbose=2)
            del train_ds, test_ds
            gc.collect()
            the_model.save_weights(cfg.save_mlp_dir(fold))

            ave_person_score, pos_ave_person_score = metrics_ave_time_id(test_target, the_model.predict(test_ox, batch_size=2048))
            print('\n Ave_Person_Score: {0}\n Positive_Person_Score: {1}\n'.format(ave_person_score, pos_ave_person_score))
            print('**************************************************************************************************\n')
            K.clear_session()
            if not cfg.use_time_kfold:
                if fold == cfg.simple_kfold_get:
                    break
